In [4]:
#!conda create -n MediaPipe python=3.9 -y
#!conda run -n MediaPipe pip install mediapipe ipykernel
#!conda run -n MediaPipe python -m ipykernel install --user --name=MediaPipe --display-name "Python (MediaPipe)"

In [5]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2 #to read and process images
import matplotlib.pyplot as plt #to show resultant images 

import cv2 #to read and process images
import matplotlib.pyplot as plt #to show resultant images 

In [6]:
import cv2
import mediapipe as mp

# Setup
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Open video
cap = cv2.VideoCapture('clean_helland.mp4')  # or 'clean_helland.mp4'

# Set display window size
DISPLAY_WIDTH = 960
DISPLAY_HEIGHT = 540

# Setup MediaPipe Pose
with mp_pose.Pose(static_image_mode=False,
                  model_complexity=1,
                  smooth_landmarks=True,
                  enable_segmentation=False,
                  min_detection_confidence=0.5,
                  min_tracking_confidence=0.5) as pose:

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Process original frame (don't resize here)
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image_rgb.flags.writeable = False

        results = pose.process(image_rgb)

        image_rgb.flags.writeable = True
        output_frame = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

        # Draw landmarks
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                output_frame,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
            )

        # Resize *for display only*, maintaining aspect ratio
        h, w = output_frame.shape[:2]
        aspect_ratio = w / h
        if w > h:
            new_w = DISPLAY_WIDTH
            new_h = int(DISPLAY_WIDTH / aspect_ratio)
        else:
            new_h = DISPLAY_HEIGHT
            new_w = int(DISPLAY_HEIGHT * aspect_ratio)

        resized_output = cv2.resize(output_frame, (new_w, new_h))

        # Show frame
        cv2.imshow('Pose Estimation (fit to screen)', resized_output)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
